In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


### Load webpage, find the table
### replace "Not assign" values in Borough column by Nan

In [51]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df["Borough"].replace("Not assigned", np.nan, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Print Nan values within each column (True means missing)

In [52]:
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Postcode
False    288
Name: Postcode, dtype: int64

Borough
False    211
True      77
Name: Borough, dtype: int64

Neighbourhood
False    288
Name: Neighbourhood, dtype: int64



### drop every row with NaN in "Borough" column and check again

In [39]:
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped rows
df.reset_index(drop=True, inplace=True)
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

Postcode
False    211
Name: Postcode, dtype: int64

Borough
False    211
Name: Borough, dtype: int64

Neighbourhood
False    211
Name: Neighbourhood, dtype: int64



### agregate Neighborhoods and replace Not assigned neigborhood to Borough value
### change initial NeighboUrhood to Neighborhood

In [55]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.loc[df.Neighbourhood == 'Not assigned', "Neighbourhood"] = df.Borough

for index, row in df.iterrows():
    if(row['Neighbourhood'] == 'Not assigned'):
        print(row['Borough'], row['Neighbourhood'])
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [58]:
print("number of rows: ",df.shape[0])

number of rows:  103
